In [1]:
################################# BLOCK 1 : 공통 ################################### 

# 필요한 모듈과 라이브러리를 로딩
import pandas as pd    
import pymysql

################################# BLOCK 2 : 크롤러 ################################# 

#1단계에선 크롤러 없이 리스트에 값을 직접 넣어서 수행합니다~

############################# BLOCK 3 : 데이터 정리 ################################

# 데이터베이스에 들어갈 값을 임의로 지정합니다. 추후 크롤러가 대체합니다.
test_df = pd.DataFrame()
test_df['판매순위']=[1,2,3]
test_df['제품소개']=['콜라','사이다','웰치스']
test_df['제품판매가']=['100','200','300']
test_df['상품평수']=[123,234,456]
test_df['상품평점']=[5.0,4.5,4.8]
test_df['분류']=['탄산음료','탄산음료','탄산음료']

################################# BLOCK 4 : mySQL ################################# 

# mySQL과 연결하기 위한 필수값 설정
host_name = "localhost"     # DB가 존재하는 host, localhost가 아닌 ip의 경우 사전에 권한을 주어야 합니다.
username = "root"           # 해당 DB의 USER NAME을 적습니다.(mySQL editor에 들어가는 계정의 이름)
password = "oracle"         # 해당 USER의 PASSWORD를 적습니다.(mySQL editor에 접속할 때 사용하는 비밀번호)
database_name = "mysql"     # 연결할 데이터베이스의 이름

# mySQL과 연결
con = pymysql.connect(host=host_name,port=3306,user=username, passwd=password, db=database_name,charset='utf8')
cur = con.cursor()

# # # # # 데이터베이스 및 테이블 구성 # # # # #
# STEP1. IF EXISTS 구문을 활용해 같은 이름의 DB가 존재하면 없애고(DROP), 추후(STEP2) 새로 만든다.
SQL_QUERY = """ DROP DATABASE IF EXISTS test_db ; """
cur.execute(SQL_QUERY)

# STEP2. 데이터베이스 생성
SQL_QUERY = """ CREATE DATABASE test_db ; """
cur.execute(SQL_QUERY)

# STEP3. 생성한 데이터 베이스를 사용하는 데이터 베이스로 지정
SQL_QUERY=""" USE test_db ; """
cur.execute(SQL_QUERY)

# STEP4. 활성화된 데이터 베이스 안에 지정한 이름의 테이블이 중복 존재하는 경우 삭제(DROP)
# 지정형식은 DB.TABLE, 여기서는 test_db 아래의 test라는 이름의 테이블을 조회한 후 중복존재하면 삭제하는 것
SQL_QUERY=""" DROP TABLE IF EXISTS test_db.test ; """
cur.execute(SQL_QUERY)

# STEP5. 테이블 생성
# VARCHAR2는 VARCHAR로, NUMBER는 정수의 경우 INT로, 소수점이 있는 경우 DECIMAL로 바꿔줍니다.
# CRAWLED 값의 경우엔 추후에 따로 SYSDATE 값을 넣지 않고 테이블 생성 과정에서 DATETIME과 DEFAULT 값으로 설정해줍니다.

SQL_QUERY=""" 
    CREATE TABLE test_db.test (
    RANKING INT, 
    TITLE VARCHAR(256) ,
    PRICE VARCHAR(10) ,
    REVIEW INT ,
    STARS DECIMAL(2,1) ,
    CATEGORY VARCHAR(20) ,
    CRAWLED DATE DEFAULT (CURRENT_DATE)     
) DEFAULT CHARSET=utf8 COLLATE=utf8_bin 
"""   # 앞에 DATE를 DATETIME으로 쓰면 시간까지 표시되고, DATE라고만 하면 날짜만 나옵니다.
cur.execute(SQL_QUERY)
con.commit()

# STEP6. 테이블에 들어갈 값을 지정합니다.
# 첫 괄호에는 각각의 항목에 부여할 열의 이름을, 그 다음에는 그 데이터에 걸맞은 형식을 지정해줍니다.
# mySQL의 경우에는 VALUE 값을 일일히 순서로 지정하지 않고, %s로 적은 뒤 for구문과 executemany를 이용해 하나씩 대입시킵니다.
sql = ''' INSERT INTO test_db.test (RANKING, TITLE, PRICE, REVIEW, STARS, CATEGORY)
        VALUES (%s,%s,%s,%s,%s,%s) ''' 

rows = [tuple(x) for x in test_df.values]
cur.executemany(sql,rows)
con.commit()

# sql=''' SELECT * FROM test_db.test '''
# cur2=con.cursor()
# cur2.execute(sql)
# cur2.fetchall() 